# Install, Paths and Parameters

In [ ]:
# Requirements. Need to restart after installation (it sucks, I am sorry haha)
!pip install torch==1.7.1
!pip install torchvision==0.8.2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import os
import pandas as pd
import numpy as np
import json
import torch
from typing import List, Callable
import random

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

In [4]:
# general path
DRIVE_PATH = '/content/drive/MyDrive/'
DL_PROJECT_PATH = os.path.join(DRIVE_PATH, 'DeepLearningProject')
ADV_PATH = os.path.join(DL_PROJECT_PATH, 'AdversarialAttacks')

# filenames and label path
ADV_LABEL_PATH = os.path.join(ADV_PATH,'adv_data/val_damagenet_cl.txt')
ORG_LABEL_PATH = os.path.join(ADV_PATH,'ori_data/correct_labels_cl.txt')

# image paths
ORIGINAL_IMAGES_PATH = os.path.join(ADV_PATH,'ori_data/ImageNetClasses/')
ADVERSARIAL_IMAGES_PATH = os.path.join(ADV_PATH,'adv_data/DAmageNetClasses/')

# attention paths
ORG_ATTN_PATH = os.path.join(ADV_PATH,'org_attn/')
ADV_ATTN_PATH = os.path.join(ADV_PATH,'adv_attn/')

In [39]:
def get_random_classes(number_of_classes: int = 50, min_rand_class: int = 0, max_rand_class: int = 999):
  return np.random.randint(low=min_rand_class, high=max_rand_class, size=(number_of_classes,))

CLASS_SUBSET = get_random_classes()

BATCH_SIZE = 6 # You can play around with it

# Import DINO
Official repo: https://github.com/facebookresearch/dino

In [6]:
# Load pretrained weights from PyTorch
device = 'cuda'
vits16 = torch.hub.load('facebookresearch/dino:main', 'dino_vits8').to(device)

Downloading: "https://github.com/facebookresearch/dino/archive/main.zip" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_deitsmall8_pretrain/dino_deitsmall8_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_deitsmall8_pretrain.pth


  0%|          | 0.00/82.7M [00:00<?, ?B/s]

In [7]:
# Hyperparameters
N_LAST_BLOCKS = 4 # Took from official repo
PATCH_SIZE=8

In [40]:
# Define and load pretrained weights for linear classifier on ImageNet
from torch import nn
class LinearClassifier(nn.Module):
    """Linear layer to train on top of frozen features"""
    def __init__(self, dim, num_labels=1000):
        super(LinearClassifier, self).__init__()
        self.num_labels = num_labels
        self.linear = nn.Linear(dim, num_labels) 
        self.linear.weight.data.normal_(mean=0.0, std=0.01)
        self.linear.bias.data.zero_()

    def forward(self, x):
        # flatten
        x = x.view(x.size(0), -1)

        # linear layer
        return self.linear(x)

linear_classifier = LinearClassifier(vits16.embed_dim * N_LAST_BLOCKS, num_labels=1000)
linear_classifier = linear_classifier.cuda()

linear_state_dict = torch.hub.load_state_dict_from_url(url="https://dl.fbaipublicfiles.com/dino/" + "dino_deitsmall8_pretrain/dino_deitsmall8_linearweights.pth")["state_dict"]

# Update state dict to avoid crash. Workaround.
linear_state_dict['linear.weight'] = linear_state_dict.pop('module.linear.weight')
linear_state_dict['linear.bias'] = linear_state_dict.pop('module.linear.bias')

# Load pre-trained weights
linear_classifier.load_state_dict(linear_state_dict, strict=True)

<All keys matched successfully>

# Load data

In [81]:
# Custom loader
from torch.utils.data import Dataset
from torchvision.io import read_image
import traceback
from PIL import Image

class ImageDataset(Dataset):
  def __init__(self, img_folder: str, file_name: str, transform: callable, class_subset: List[int] = None):
    super().__init__()
    self.transform=transform
    self.img_folder=img_folder
    self.data = self.create_df(file_name)
    self.class_subset = class_subset
    if self.class_subset is None:
      self.data_subset = self.data
    else:
      self.data_subset = self.data[self.data['label'].isin(self.class_subset)]
  
  def create_df(self, file_name: str):
    df = pd.read_csv(file_name, sep=" ", header=None)
    df.columns=['file', 'label']
    return df
    
  def __len__(self):
    return len(self.data_subset)
  
  def __getitem__(self, index):
    img = Image.open(os.path.join(self.img_folder,self.data_subset['file'].iloc[index]))
    img = img.convert('RGB')

    img=self.transform(img)
    target=self.data_subset['label'].iloc[index]

    return img,target,self.data_subset['file'].iloc[index]


In [82]:
from torchvision import transforms as pth_transforms

# Create loader
# Taken from official repo: https://github.com/facebookresearch/dino/blob/main/eval_linear.py
transform = pth_transforms.Compose([
    pth_transforms.Resize(256, interpolation=3),
    pth_transforms.CenterCrop(224),
    pth_transforms.ToTensor(),
    pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

adv_dataset = ImageDataset(img_folder = ADVERSARIAL_IMAGES_PATH, 
                           file_name = ADV_LABEL_PATH,
                           transform=transform,
                           class_subset=CLASS_SUBSET)

org_dataset = ImageDataset(img_folder = ORIGINAL_IMAGES_PATH,
                           file_name = ORG_LABEL_PATH,
                           transform=transform,
                           class_subset=CLASS_SUBSET)

org_loader = torch.utils.data.DataLoader(
    org_dataset,
    batch_size=BATCH_SIZE,
    num_workers=1,
    pin_memory=True,
)

adv_loader = torch.utils.data.DataLoader(
    adv_dataset,
    batch_size=BATCH_SIZE,
    num_workers=1,
    pin_memory=True,
)

Important note: adversarial labels contain the actual class for the image. Therefore, attack succeeds if prediction is different from it.

# Adversarial dataset generation (Drive)

- original sample is correctly classified by DINO
- their adversarial attack actually changes the prediction

In [70]:
import torch

# Performs a forward pass given a sample `inp` and a classifier.
@torch.no_grad()
def forward_pass(inp, model, classifier, n):
  with torch.no_grad():
    intermediate_output = model.get_intermediate_layers(inp, n)
    output = torch.cat([x[:, 0] for x in intermediate_output], dim=-1)
    output = classifier(output)
    return output.argmax(1)

In [83]:
# A Python generator for adversarial samples.
# Takes original and adversarial loaders, a model, classifier and yields
# a pair of original and adversarial samples based on the definition above.
def adv_dataset(org_loader, adv_loader, model, classifier=linear_classifier, n=N_LAST_BLOCKS):
  linear_classifier.eval()
  model.eval()
  for org, adv in zip(org_loader, adv_loader):
    # parse the original sample
    org_inp, org_tar, org_img_name = org
    org_inp = org_inp.to(device)
    org_tar = org_tar.to(device)

    # parse the adversarial sample
    adv_inp, adv_tar, adv_img_name = adv
    adv_inp = adv_inp.to(device)
    adv_tar = adv_tar.to(device)

    # forward pass original and adversarial sample
    org_pred = forward_pass(org_inp, model, linear_classifier, n)
    adv_pred = forward_pass(adv_inp, model, linear_classifier, n)

    #print(f"Original prediction: {org_pred}")
    #print(f"Adversarial prediction: {adv_pred}")
    #print(f"Correct label: {org_tar}")
    #print("\n")
    
    # label, original image predicted class, adversarial image predicted class
    for y, org_y, adv_y, org_x, adv_x, org_name, adv_name in zip(org_tar, org_pred, adv_pred, org_inp, adv_inp, org_img_name, adv_img_name):
      # yield a new tuple based if the conditions match. skip otherwise.
      org_correct = y == org_y
      adv_correct = y == adv_y

      org_num = int(org_name.replace('.JPEG', '').split("_")[-1])
      adv_num = int(adv_name.replace('.png', '').split("_")[-1])
      assert org_num == adv_num, "Numbers are not matching"

      if org_correct and not adv_correct:
        yield org_num, org_x, adv_x

Optionally: save self-attention for further processing

In [ ]:
import sys

# output first 5 tuples generated.
samples = adv_dataset(org_loader, adv_loader, vits16, linear_classifier)
total=1000

for i in range(total):
  print(i)
  num, org, adv = next(samples)
  sys.stdout.write(f"\rtuple {i+1}/{total}")
  sys.stdout.flush()

  # self attention
  # add one dimension to input image (get_last_selfattention expects it)
  org = org.unsqueeze(0)
  adv = adv.unsqueeze(0)
  org_attn = vits16.get_last_selfattention(org)
  adv_attn = vits16.get_last_selfattention(adv)
  
  #torch.save(org_attn, ORG_ATTN_PATH + f"{num}.pt")
  #torch.save(adv_attn, ADV_ATTN_PATH + f"{num}.pt")

  # folders for org and adv, filename: {org, adv}/<original number>.pt (leave out prefix)

# Embedding generation

In [48]:
ORG_EMB_PATH = os.path.join(ADV_PATH,'org_emb_mean_patches/')
ADV_EMB_PATH = os.path.join(ADV_PATH,'adv_emb_mean_patches/')
print(ORG_EMB_PATH)
print(ADV_EMB_PATH)

/content/drive/MyDrive/DeepLearningProject/AdversarialAttacks/org_emb_mean_patches/
/content/drive/MyDrive/DeepLearningProject/AdversarialAttacks/adv_emb_mean_patches/


In [85]:
drive.mount('/content/drive')
!rm /content/drive/MyDrive/DeepLearningProject/AdversarialAttacks/org_emb_mean_patches/*
!rm /content/drive/MyDrive/DeepLearningProject/AdversarialAttacks/adv_emb_mean_patches/*

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
def generate_model_output(inp, model, n=N_LAST_BLOCKS, target_tokens="CLS"):
    inp = inp[:, :, :].unsqueeze(0)
    intermediate_output = model.get_intermediate_layers(inp, n)
    if target_tokens == "CLS":
      return torch.cat([x[:, 0] for x in intermediate_output], dim=-1)
    elif target_tokens == "patches_mean":
      return torch.cat([x[:, 1:].mean(1) for x in intermediate_output], dim=-1)
    else:
      raise Exception("invalid target_tokens")

In [ ]:
import sys

# output first 5 tuples generated.
samples = adv_dataset(org_loader, adv_loader, vits16, linear_classifier)

# generator for cls embeddings
def cls_embed_gen(samples):
  for num, org, adv in samples:
    org_out = generate_model_output(org, vits16, target_tokens="CLS")
    adv_out = generate_model_output(adv, vits16, target_tokens="CLS")
    yield num, org_out, adv_out

# generator for mean of all patch embeddings
def mean_patches_gen(samples):
  for num, org, adv in samples:
    org_out = generate_model_output(org, vits16, target_tokens="patches_mean")
    adv_out = generate_model_output(adv, vits16, target_tokens="patches_mean")
    yield num, org_out, adv_out

# generator for variance of patch embeddings
def var_gen(samples):
  for num, org, adv in samples:
    org_out = generate_model_output(org, vits16, target_tokens="patches_mean")
    adv_out = generate_model_output(adv, vits16, target_tokens="patches_mean")
    yield num, org_out, adv_out

# Adversarial Classifier

In [ ]:
from os import listdir
import sys
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [ ]:
class AdverserialAttentionDataset(Dataset):
    """Adverserial Attention dataset."""

    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with all the .pt files that contain
                both attention of the original img and of the adv. img.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        names_org = [f[:-3]+"_org" for f in listdir(root_dir)]
        names_adv = [f[:-3]+"_adv" for f in listdir(root_dir)]
        self.n = len(names_org)
        self.file_names = names_org + names_adv
        self.transform = transform

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        attn_file = self.file_names[idx]
        adv_idx = 0
        if attn_file[-3:0] == "adv":
          adv_idx = 1
        attn_file = attn_file[:-4]
        
        attn_path = os.path.join(self.root_dir, attn_file+".pt")
        attention = torch.load(attn_path)
      
        sample = {'attention': attention[adv_idx].squeeze(), 'adverserial': adv_idx}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
# test load attention and visualize
import sys

ATTN_DIR = "/content/drive/MyDrive/Deep Learning Project/AdversarialAttacks/attentions/"

test = torch.load(ATTN_DIR+f"{0}.pt")
print(test[0].shape)

attn_dataset = AdverserialAttentionDataset(root_dir=ATTN_DIR)


In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# # Ignore warnings
# import warnings
# warnings.filterwarnings("ignore")

# plt.ion()   # interactive mode

# fig = plt.figure(figsize=(16, 12), dpi=80)

# for i in range(10,len(attn_dataset)):
#     i -= 10
#     sample = attn_dataset[i]
#     # print(sample)

#     # print(i, sample['attention'].shape, sample['adverserial'])

#     ax = plt.subplot(1, 4, i + 1)
#     plt.tight_layout()
#     ax.set_title('Sample #{}'.format(i))
#     ax.axis('off')
#     cpu_tensor = sample['attention'].cpu().detach().squeeze()
    
#     # plot all the attentions on top of eachother
#     for l in range(6):
#       plt.imshow(cpu_tensor.numpy()[l,:,:])
#     if i == 3:
#       plt.show()
#       break



In [ ]:
BATCH_SIZE = 4

# Training set and training loader
dataset_size = len(attn_dataset)
train_size = int(0.8 * dataset_size)
test_size = dataset_size - train_size

train_dataset, test_dataset = torch.utils.data.random_split(attn_dataset, 
                                                            [train_size, 
                                                             test_size],
                                                            generator=torch.Generator().manual_seed(42))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=0)

# Test set and test loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE,
                                         shuffle=True, num_workers=0)

classes = ('non-adversarial', 'adversarial') # Binary classifier


## Define CNN Classifier
2 CNN Layers, 3 FC Layers

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Network
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.adam(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, start=0):
        # get the inputs; data is a list of [inputs, labels]
        print(data)
        print(data['attention'].shape)
        print(data['adverserial'].shape)
        inputs, labels = data
        print(inputs)
        attentions = attentions[0, :, 0, 1:].reshape(nh, -1)
        attentions = attentions.reshape(nh, w_featmap, h_featmap)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
PATH = './simple_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

# Test inference

In [ ]:
# Adapted from official repo
@torch.no_grad()
def test_network(data_loader, model, linear_classifier, n=N_LAST_BLOCKS):
  linear_classifier.eval()
  model.eval()

  predictions = []

  for inp, tar in data_loader:
    inp = inp.to(device)
    tar = tar.to(device)

    print(tar)

    # forward
    with torch.no_grad():
      intermediate_output = model.get_intermediate_layers(inp, n)
      output = torch.cat([x[:, 0] for x in intermediate_output], dim=-1)
      output = linear_classifier(output)
      predictions += output.argmax(1).tolist()
      print(output.argmax(1))
      print("\n")

  return predictions

In [ ]:
predictions = test_network(org_loader, vits16, linear_classifier)

In [ ]:
predictions = test_network(adv_loader, vits16, linear_classifier)

# Visualize attention
Taken from: https://github.com/facebookresearch/dino/blob/main/visualize_attention.py

In [ ]:
import random
import colorsys
import requests
from io import BytesIO

import skimage.io
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as pth_transforms
import numpy as np
from PIL import Image

import utils
import vision_transformer as vits


def apply_mask(image, mask, color, alpha=0.5):
    for c in range(3):
        image[:, :, c] = image[:, :, c] * (1 - alpha * mask) + alpha * mask * color[c] * 255
    return image


def random_colors(N, bright=True):
    """
    Generate random colors.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors


def display_instances(image, mask, fname="test", figsize=(5, 5), blur=False, contour=True, alpha=0.5):
    fig = plt.figure(figsize=figsize, frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax = plt.gca()

    N = 1
    mask = mask[None, :, :]
    # Generate random colors
    colors = random_colors(N)

    # Show area outside image boundaries.
    height, width = image.shape[:2]
    margin = 0
    ax.set_ylim(height + margin, -margin)
    ax.set_xlim(-margin, width + margin)
    ax.axis('off')
    masked_image = image.astype(np.uint32).copy()
    for i in range(N):
        color = colors[i]
        _mask = mask[i]
        if blur:
            _mask = cv2.blur(_mask,(10,10))
        # Mask
        masked_image = apply_mask(masked_image, _mask, color, alpha)
        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        if contour:
            padded_mask = np.zeros((_mask.shape[0] + 2, _mask.shape[1] + 2))
            padded_mask[1:-1, 1:-1] = _mask
            contours = find_contours(padded_mask, 0.5)
            for verts in contours:
                # Subtract the padding and flip (y, x) to (x, y)
                verts = np.fliplr(verts) - 1
                p = Polygon(verts, facecolor="none", edgecolor=color)
                ax.add_patch(p)
    ax.imshow(masked_image.astype(np.uint8), aspect='auto')
    fig.savefig(fname)
    print(f"{fname} saved.")
    return

PATCH_SIZE = 8
THRESHOLD = 0.5
OUTPUT_DIR = "/content/drive/MyDrive/TEST"
INPUT_IMAGE = "/content/drive/MyDrive/DeepLearningProject/AdversarialAttacks/ori_data/images/ILSVRC2012_val_00000130.JPEG"
model = vits16

img = Image.open(INPUT_IMAGE)
img = img.convert('RGB')

transform = pth_transforms.Compose([
    pth_transforms.Resize(256),
    pth_transforms.ToTensor(),
    pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

img = transform(img)

# make the image divisible by the patch size
w, h = img.shape[1] - img.shape[1] % PATCH_SIZE, img.shape[2] - img.shape[2] % PATCH_SIZE
img = img[:, :w, :h].unsqueeze(0)

w_featmap = img.shape[-2] // PATCH_SIZE
h_featmap = img.shape[-1] // PATCH_SIZE

print(img.shape)
attentions = model.get_last_selfattention(img.to(device))

nh = attentions.shape[1] # number of heads

# we keep only the output patch attention
attentions = attentions[0, :, 0, 1:].reshape(nh, -1)

attentions = attentions.reshape(nh, w_featmap, h_featmap)
attentions = nn.functional.interpolate(attentions.unsqueeze(0), scale_factor=PATCH_SIZE, mode="nearest")[0].detach().cpu().numpy()

# save attentions heatmaps
os.makedirs(OUTPUT_DIR, exist_ok=True)
torchvision.utils.save_image(torchvision.utils.make_grid(img, normalize=True, scale_each=True), os.path.join(OUTPUT_DIR, "img.png"))
for j in range(nh):
    fname = os.path.join(OUTPUT_DIR, "attn-head" + str(j) + ".png")
    plt.imsave(fname=fname, arr=attentions[j], format='png')
    plt.imshow(attentions[j])
    plt.show()
    print(f"{fname} saved.")

# Sample forward pass

In [ ]:
PATCH_SIZE = 16
INPUT_IMAGE = "/content/drive/MyDrive/DeepLearningProject/AdversarialAttacks/adversarial_data/images/ILSVRC2012_val_00000130.png"
model = vits16

img = Image.open(INPUT_IMAGE)
img = img.convert('RGB')

transform = pth_transforms.Compose([
    pth_transforms.Resize(256),
    pth_transforms.ToTensor(),
    pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

img = transform(img)

# make the image divisible by the patch size
w, h = img.shape[1] - img.shape[1] % PATCH_SIZE, img.shape[2] - img.shape[2] % PATCH_SIZE
img = img[:, :w, :h].unsqueeze(0)

with torch.no_grad():
  intermediate_output = model.get_intermediate_layers(img.to(device), N_LAST_BLOCKS)
  output = torch.cat([x[:, 0] for x in intermediate_output], dim=-1)
  output = linear_classifier(output)
  print(output.argmax(1))

# Random code

In [ ]:
# Use to create understandable groundtruth for ImageNET
with open("/content/drive/MyDrive/Deep Learning Project/Adversarial Attacks/original_data/full_custom_groundtruth.txt", "w") as f_write:
  with open("/content/drive/MyDrive/Deep Learning Project/Adversarial Attacks/original_data/FULL_ILSVRC2012_validation_ground_truth.txt", "r") as f:
    for i, line in enumerate(f):
      f_write.write(f"ILSVRC2012_val_{i+1:08}.JPEG "+line)

# Local copy

Option to download all data instead of using GDrive mount

ImageNet

In [ ]:
%%bash
gdown --id 1ZoVlgzkFD9cyv6jdenQ7JUVuS9sXMpEG
mkdir -p ImageNet
tar -xvf ILSVRC2012_img_val.tar -C ImageNet
rm ILSVRC2012_img_val.tar

DAmageNet

In [ ]:
%%bash
gdown --id 1K2C38bWRCeuNRtWW6qbAF8xFjxB-969U
unzip "/content/DAmageNet.zip"
rm DAmageNet.zip

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Downloading...
From: https://drive.google.com/uc?id=1K2C38bWRCeuNRtWW6qbAF8xFjxB-969U
To: /content/DAmageNet.zip
100%|##########| 5.46G/5.46G [01:57<00:00, 46.6MB/s]


In [ ]:
ORIGINAL_IMAGES_PATH = '/content/ImageNet'
ADVERSARIAL_IMAGES_PATH = '/content/DAmageNet/DAmageNet'
!mkdir -p '/content/org_attn/'
!mkdir -p '/content/adv_attn/'

In [ ]:
# alternatively set attention path local as well
ORG_ATTN_PATH = '/content/org_attn/'
ADV_ATTN_PATH = '/content/adv_attn/'